# laserscans 

In [ ]:
from turtle import Screen
import boto3, os.path, yaml , glob
from boto3.s3.transfer import TransferConfig

paswd_file = os.path.join( r'F:\secrets' , 'config.yml')
with open(paswd_file) as f:
    pass_dict = yaml.safe_load(f)

## acc-laserscans
END_POINT = "https://s3-ant1.antwerpen.be"
ACC_CLIENT_KEY = pass_dict['kw']['S3_LASER']['CLIENT_KEY'] 
ACC_SECRET =  pass_dict['kw']['S3_LASER']['SECRET'] 
ACC_TARGET = "acc-laserscans"

CLIENT_KEY = pass_dict['kw']['S3_LASER_PROD']['CLIENT_KEY'] 
SECRET =  pass_dict['kw']['S3_LASER_PROD']['SECRET'] 
TARGET = "prod-laserscans"


## create new bucket

In [ ]:
session = boto3.session.Session(aws_access_key_id= CLIENT_KEY, 
                                    aws_secret_access_key= SECRET)
s3 = session.resource("s3", endpoint_url= END_POINT)
s3_client = session.client('s3', endpoint_url= END_POINT)
#s3_client.create_bucket(Bucket="acc-antwerpen-herdenkt")
s3_client

## set CORS

In [ ]:
#SET cors settings : 
#https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-example-configuring-buckets.html
bucket = s3.Bucket(TARGET)
cors_configuration = {
    'CORSRules': [{
        'AllowedHeaders': ['*'],
        'AllowedMethods': ['GET', 'PUT', 'HEAD'],
        'AllowedOrigins': ['*'],
        'ExposeHeaders': ['GET', 'PUT', 'HEAD', 'ETag', 'x-amz-meta-myheader'],
        'MaxAgeSeconds': 3000
    }]
}
s3_client.put_bucket_cors(Bucket= TARGET , CORSConfiguration=cors_configuration)
s3_client.get_bucket_cors(Bucket= TARGET)

In [ ]:
os.path.basename( glob.glob( "../llvv/*.geojson")[1] )

## upload data

In [ ]:
bucket = s3.Bucket(TARGET)
gfiles = [ os.path.abspath(n) for n in glob.glob( "../data/*.geojson") ]

for f in gfiles:
    targetF = 'data/'+ os.path.basename(f)
    bucket.put_object(Key=targetF, 
                      Body=open(f , 'rb'), 
                      ACL='public-read', ContentType='application/json' ) 
                      
vfiles = [ os.path.abspath(n) for n in glob.glob( "../data/*.svg") ]
for f in vfiles:
    targetF = 'data/'+ os.path.basename(f)
    bucket.put_object(Key=targetF, 
                      Body=open(f , 'rb'), 
                      ACL='public-read', ContentType='image/svg+xml' ) 

In [ ]:
list(bucket.objects.all())